In [2]:
library(opploansanalytics)
load.packages()

RPostgreSQL    RPostgres        RJDBC         httr     jsonlite googlesheets 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
   tidyverse        tidyr        dplyr    lubridate      stringr      ggplot2 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
      ggpubr      cowplot        rJava        mailR     devtools     roxygen2 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE

In [ ]:
df = queryReporting(
"
select
    --  Identifiers  --
    lde.lead_id
    , lde.partnerid
    , lde.leadofferid
    , lde.lead_time
    , c_am.name
    , lde.accepted
    , lde.reason
    , lde.offer_amount
    --  Banking  --
    , lde.abaroutingnumber
    , lde.accountnumber
    , lde.bankname
    --  Employment  --
    , lde.grossmonthlyincome
    , lde.incometype
    , lde.lastpayrolldate
    , lde.payrollfrequency
    , lde.payrolltype
    --  Contact  --
    , lde.city
    , lde.address
    , lde.dateofbirth
    , lde.statecode
    , lde.raw_lead
from
    lde4.leads as lde
    inner join
        cloudlending.advertising_method as c_am
        on lde.partnerid = c_am.external_id
        and c_am.name = 'LenderEdge 4'
"
)